In [217]:
import os
import datetime
import pandas as pd
import numpy as np

from bokeh.plotting import figure, show, output_file
from bokeh.models import (ColumnDataSource,
                          DatetimeTickFormatter,
                          LinearAxis,
                          Range1d, 
                          NumeralTickFormatter,
                          HoverTool,
                          Legend,
                          LegendItem)


In [229]:
# set the filepath 
username = os.getlogin()

# read in data
df = pd.read_stata(os.path.join("/scratch", f"{username}", "agmark_total_ts.dta"))

# define annualized date variable
df['month'] = df['date'].dt.month.astype(str)
df['day'] = df['date'].dt.day.astype(str)
df["month-day"] = df["month"] + "-" + df["day"] + "-" + "2016"
df["annualized"] = df["month-day"].apply(lambda x: datetime.datetime.strptime(x, "%m-%d-%Y"))

# take 3-day rolling average
df['qty_rolling'] = df['qty'].rolling(window=3).mean()

# set index to be the date
df.index = df["date"]
df.index.name = 'Date'
df.sort_index(inplace=True)

# extract data for each year
df_2018 = pd.DataFrame(df.loc[df['year'] == 2018].set_index("annualized")['qty_rolling'])
df_2018 = df_2018.rename(columns={"qty_rolling": "qty_2018"})
df_2019 = pd.DataFrame(df.loc[df['year'] == 2019].set_index("annualized")['qty_rolling'])
df_2019 = df_2019.rename(columns={"qty_rolling": "qty_2019"})
data = df.loc[df['year'] == 2020].set_index("annualized")
data = data.rename(columns={"qty_rolling": "qty_2020"})

# merge all years to stack the data
data = data.merge(df_2018, left_index=True, right_index=True)
data = data.merge(df_2019, left_index=True, right_index=True)

# calculate the percentage change between 2019 and 2020
data['perc_change_qty'] = (data['qty_2020'] - data['qty_2019']) / data['qty_2019']

# create the date as a string variable
data["date_str"] = data["date"].apply(lambda x: x.strftime("%d-%b"))

# replace 0 values with nan
data = data.replace({"qty_2020": {0: np.nan}})

In [223]:
# define bokeh-source data
source = ColumnDataSource(data)

# define tools
TOOLS = "pan,box_zoom,reset,lasso_select,save,box_select,xzoom_in,crosshair"

# define bokey figure
p = figure(tools=TOOLS,
           x_axis_type="datetime",
           plot_width=1000,
           plot_height=600,
           toolbar_location="right",
           y_axis_label="India-Wide Food Arrivales (Tonnes)")

# set the second y axis range name and range
p.extra_y_ranges = {"cases": Range1d(start=0, end=np.max(data['cases'])+5)}

# add the second axis to the plot.  
p.add_layout(LinearAxis(y_range_name="cases", axis_label="Number of New Cases"), 'right')

# plot new covid cases on second y axis
p.vbar(x='annualized',
       top='cases',
       width=60*60*1000*24*0.75,
       alpha=0.65,
       color="#ffbb00",
       source=source,
       y_range_name="cases")

# plot the quantity data for each year
p.line('annualized', 'qty_2018', source=source, color="#077300", line_width=1.5, line_dash="dotdash")
p.line('annualized', 'qty_2019', source=source, color="#050063", line_width=1.5, line_dash="dotdash")
p.line('annualized', 'qty_2020', source=source, color="red", line_width=2)

# set axes parameters
p.x_range.start = datetime.datetime(2016,1,1)
p.x_range.end = datetime.datetime(2016,6, 4)
p.xaxis.formatter=DatetimeTickFormatter(months="%B")
p.yaxis.formatter=NumeralTickFormatter(format="0,0")
p.xaxis.axis_label = 'Date'

# define hover tip tool
hover1 = HoverTool(
    tooltips = [
        ("New Cases", "@cases{0}"),
        ("Percent Quantity Change, 2019 to 2020", "@perc_change_qty{:.2%}"),
        ("Date", "@date_str")],
)
p.add_tools(hover1)

# create legend
li1 = LegendItem(label='New Covid-19 Cases', renderers=[p.renderers[0]])
li2 = LegendItem(label='2018 Market Quantity', renderers=[p.renderers[1]])
li3 = LegendItem(label='2019 Market Quantity', renderers=[p.renderers[2]])
li4 = LegendItem(label='2020 Market Quantity', renderers=[p.renderers[3]])
legend1 = Legend(items=[li1, li2, li3, li4], location='top_left')
p.add_layout(legend1)

# output html
output_file("ts.html")
show(p)